In [ ]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa para criar uma base de empresas que têm como sócios políticos eleitos ou suplentes em 2018
#

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime
import unidecode

In [36]:
def f(str):
    return (unidecode.unidecode(str))

Dados do TSE

http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

In [37]:
candidatos_2018 = pd.read_csv("candidatos_2018/consulta_cand_2018_BRASIL.csv",\
                              sep=';', \
                              encoding='latin_1', \
                              converters={'NR_CPF_CANDIDATO': lambda x: str(x),
                                          'SQ_CANDIDATO': lambda x: str(x)},\
                              usecols=['NR_CPF_CANDIDATO',
                                       'NM_CANDIDATO',
                                       'ANO_ELEICAO',
                                       'SQ_CANDIDATO',
                                       'DS_CARGO',
                                       'SG_UF', 
                                       'DS_SIT_TOT_TURNO'])

In [38]:
candidatos_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 7 columns):
ANO_ELEICAO         29145 non-null int64
SG_UF               29145 non-null object
DS_CARGO            29145 non-null object
SQ_CANDIDATO        29145 non-null object
NM_CANDIDATO        29145 non-null object
NR_CPF_CANDIDATO    29145 non-null object
DS_SIT_TOT_TURNO    29145 non-null object
dtypes: int64(1), object(6)
memory usage: 1.6+ MB


In [39]:
candidatos_2018.reset_index().head()

,index,ANO_ELEICAO,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,DS_SIT_TOT_TURNO
0,0,2018,SE,DEPUTADO ESTADUAL,260000629008,INGRID BITENCOURT,01478793538,#NULO#
1,1,2018,AM,DEPUTADO ESTADUAL,40000611557,AURELIANO GOMES JUNIOR,62983628272,SUPLENTE
2,2,2018,SP,DEPUTADO FEDERAL,250000623920,PAOLLA CATHERINE DOS SANTOS MIGUEL AGARD,22857846860,SUPLENTE
3,3,2018,AL,DEPUTADO ESTADUAL,20000628963,VERONICA DA SILVA,06779642428,SUPLENTE
4,4,2018,PA,DEPUTADO ESTADUAL,140000612785,RENATO ALVES SAMPAIO,57782709253,SUPLENTE


In [40]:
candidatos_2018['DS_SIT_TOT_TURNO'].value_counts()

SUPLENTE            18666
NÃO ELEITO           6927
#NULO#               1702
ELEITO POR QP        1275
ELEITO POR MÉDIA      297
ELEITO                218
2º TURNO               60
Name: DS_SIT_TOT_TURNO, dtype: int64

In [41]:
candidatos_2018 = candidatos_2018[(candidatos_2018['DS_SIT_TOT_TURNO'] == 'SUPLENTE') |
                                  (candidatos_2018['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP') |
                                  (candidatos_2018['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') |
                                  (candidatos_2018['DS_SIT_TOT_TURNO'] == 'ELEITO') 
                                 ]

In [42]:
candidatos_2018.reset_index().head()

,index,ANO_ELEICAO,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,DS_SIT_TOT_TURNO
0,1,2018,AM,DEPUTADO ESTADUAL,40000611557,AURELIANO GOMES JUNIOR,62983628272,SUPLENTE
1,2,2018,SP,DEPUTADO FEDERAL,250000623920,PAOLLA CATHERINE DOS SANTOS MIGUEL AGARD,22857846860,SUPLENTE
2,3,2018,AL,DEPUTADO ESTADUAL,20000628963,VERONICA DA SILVA,06779642428,SUPLENTE
3,4,2018,PA,DEPUTADO ESTADUAL,140000612785,RENATO ALVES SAMPAIO,57782709253,SUPLENTE
4,6,2018,RJ,DEPUTADO ESTADUAL,190000622928,RENAN FERREIRINHA CARNEIRO,13698925788,ELEITO POR MÉDIA


In [43]:
# Cria nova coluna com o CPF mascarado dos doadores
# 11 dígitos
# 12345678901
# 01234567890 -> python começa contar pelo zero
# ***345678**

# List slicing
# [start:end]
# inclusive:exclusive

In [44]:
candidatos_2018['cpf_mascarado']='***'+candidatos_2018['NR_CPF_CANDIDATO'].str[3:9]+'**'

In [45]:
candidatos_2018["NM_CANDIDATO"] = candidatos_2018["NM_CANDIDATO"].apply(f)

In [46]:
candidatos_2018.reset_index().head()

,index,ANO_ELEICAO,SG_UF,DS_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NR_CPF_CANDIDATO,DS_SIT_TOT_TURNO,cpf_mascarado
0,1,2018,AM,DEPUTADO ESTADUAL,40000611557,AURELIANO GOMES JUNIOR,62983628272,SUPLENTE,***836282**
1,2,2018,SP,DEPUTADO FEDERAL,250000623920,PAOLLA CATHERINE DOS SANTOS MIGUEL AGARD,22857846860,SUPLENTE,***578468**
2,3,2018,AL,DEPUTADO ESTADUAL,20000628963,VERONICA DA SILVA,06779642428,SUPLENTE,***796424**
3,4,2018,PA,DEPUTADO ESTADUAL,140000612785,RENATO ALVES SAMPAIO,57782709253,SUPLENTE,***827092**
4,6,2018,RJ,DEPUTADO ESTADUAL,190000622928,RENAN FERREIRINHA CARNEIRO,13698925788,ELEITO POR MÉDIA,***989257**


Dados da Receita Federal - maio de 2019

http://receita.economia.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-publicos-cnpj

Dados prontos para uso (limpo por https://twitter.com/turicas): https://drive.google.com/drive/folders/1uDRGu0rzUc1X-IwVlZpswtfvsUfpCOhy

Primeiro procura dados de polticos na base sócios

In [47]:
TextFileReader = pd.read_csv('receita/socio.csv',\
                             chunksize=100000,\
                             sep=',',\
                             names=['cnpj',
                                    'identificador_de_socio',
                                    'nome_socio',
                                    'cnpj_cpf_do_socio',
                                    'codigo_qualificacao_socio',
                                    'percentual_capital_social',
                                    'data_entrada_sociedade',
                                    'cpf_representante_legal',
                                    'nome_representante_legal',
                                    'codigo_qualificacao_representante_legal'],\
                             header=None,\
                             dtype={'cnpj': str,
                                    'identificador_de_socio': str,
                                    'nome_socio': str,
                                    'cnpj_cpf_do_socio': str,
                                    'codigo_qualificacao_socio': str,
                                    'percentual_capital_social': str,
                                    'data_entrada_sociedade': str,
                                    'cpf_representante_legal': str,
                                    'nome_representante_legal': str,
                                    'codigo_qualificacao_representante_legal': str
},\
                             usecols=['cnpj',
                                      'nome_socio',
                                      'cnpj_cpf_do_socio',
                                      'percentual_capital_social',
                                      'codigo_qualificacao_socio',
                                      'cpf_representante_legal',
                                      'nome_representante_legal'],
                             decimal=',')

In [48]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    polit_encontrados = pd.merge(candidatos_2018.drop_duplicates('NR_CPF_CANDIDATO'), \
                                 df, \
                                 left_on=['NM_CANDIDATO', 'cpf_mascarado'], \
                                 right_on=['nome_socio', 'cnpj_cpf_do_socio'])
    
    if polit_encontrados.empty is False:
        if conta == 0:
            df_procura1 = polit_encontrados
        else:
            df_procura1 = df_procura1.append(polit_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2019-08-08 16:46:09.794961
A extração terminou às:  2019-08-08 16:46:44.932422


In [49]:
df_procura1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21186 entries, 0 to 66
Data columns (total 15 columns):
ANO_ELEICAO                  21186 non-null int64
SG_UF                        21186 non-null object
DS_CARGO                     21186 non-null object
SQ_CANDIDATO                 21186 non-null object
NM_CANDIDATO                 21186 non-null object
NR_CPF_CANDIDATO             21186 non-null object
DS_SIT_TOT_TURNO             21186 non-null object
cpf_mascarado                21186 non-null object
cnpj                         21186 non-null object
nome_socio                   21186 non-null object
cnpj_cpf_do_socio            21186 non-null object
codigo_qualificacao_socio    21186 non-null object
percentual_capital_social    21186 non-null object
cpf_representante_legal      47 non-null object
nome_representante_legal     47 non-null object
dtypes: int64(1), object(14)
memory usage: 2.6+ MB


Depois procura esse resultados na base geral de empresas 

In [50]:
TextFileReader = pd.read_csv('receita/empresa.csv',\
                             chunksize=100000,\
                             sep=',',\
                             names=['cnpj',
                                    'identificador_matrizfilial',
                                    'razao_social',
                                    'nome_fantasia', 
                                    'situacao_cadastral',
                                    'data_situacao_cadastral',
                                    'motivo_situacao_cadastral', 
                                    'nome_cidade_exterior',
                                    'codigo_natureza_juridica',
                                    'data_inicio_atividade',
                                    'cnae_fiscal',
                                    'descricao_tipo_logradouro',
                                    'logradouro', 
                                    'numero', 
                                    'complemento', 
                                    'bairro', 
                                    'cep', 
                                    'uf', 
                                    'codigo_municipio', 
                                    'municipio', 
                                    'ddd_telefone_1',
                                    'ddd_telefone_2', 
                                    'ddd_fax', 
                                    'qualificacao_do_responsavel', 
                                    'capital_social', 
                                    'porte', 
                                    'opcao_pelo_simples', 
                                    'data_opcao_pelo_simples', 
                                    'data_exclusao_do_simples', 
                                    'opcao_pelo_mei', 
                                    'situacao_especial', 
                                    'data_situacao_especial'],\
                             header=None,\
                             dtype={'cnpj': str,
                                    'identificador_matrizfilial': str,
                                    'razao_social': str,
                                    'nome_fantasia': str, 
                                    'situacao_cadastral': str,
                                    'data_situacao_cadastral': str,
                                    'motivo_situacao_cadastral': str, 
                                    'nome_cidade_exterior': str,
                                    'codigo_natureza_juridica': str,
                                    'data_inicio_atividade': str,
                                    'cnae_fiscal': str,
                                    'descricao_tipo_logradouro': str,
                                    'logradouro': str, 
                                    'numero': str, 
                                    'complemento': str, 
                                    'bairro': str, 
                                    'cep': str, 
                                    'uf': str, 
                                    'codigo_municipio': str, 
                                    'municipio': str, 
                                    'ddd_telefone_1': str,
                                    'ddd_telefone_2': str, 
                                    'ddd_fax': str, 
                                    'qualificacao_do_responsavel': str, 
                                    'capital_social': str, 
                                    'porte': str, 
                                    'opcao_pelo_simples': str, 
                                    'data_opcao_pelo_simples': str, 
                                    'data_exclusao_do_simples': str, 
                                    'opcao_pelo_mei': str, 
                                    'situacao_especial': str, 
                                    'data_situacao_especial': str},\
                             usecols=['cnpj',
                                      'identificador_matrizfilial',
                                      'razao_social',
                                      'nome_fantasia', 
                                      'situacao_cadastral',
                                      'data_situacao_cadastral',
                                      'motivo_situacao_cadastral', 
                                      'nome_cidade_exterior',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio', 
                                      'qualificacao_do_responsavel', 
                                      'capital_social'
                                    ],\
                            decimal=',')


In [51]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    empresas_encontradas = pd.merge(df_procura1, \
                                 df, \
                                 left_on='cnpj', \
                                 right_on='cnpj')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_procura_final = empresas_encontradas
        else:
            df_procura_final = df_procura_final.append(empresas_encontradas)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2019-08-08 16:47:07.901554
A extração terminou às:  2019-08-08 16:48:54.404501


In [52]:
df_procura_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21186 entries, 0 to 39
Data columns (total 29 columns):
ANO_ELEICAO                    21186 non-null int64
SG_UF                          21186 non-null object
DS_CARGO                       21186 non-null object
SQ_CANDIDATO                   21186 non-null object
NM_CANDIDATO                   21186 non-null object
NR_CPF_CANDIDATO               21186 non-null object
DS_SIT_TOT_TURNO               21186 non-null object
cpf_mascarado                  21186 non-null object
cnpj                           21186 non-null object
nome_socio                     21186 non-null object
cnpj_cpf_do_socio              21186 non-null object
codigo_qualificacao_socio      21186 non-null object
percentual_capital_social      21186 non-null object
cpf_representante_legal        47 non-null object
nome_representante_legal       47 non-null object
identificador_matrizfilial     21186 non-null object
razao_social                   21186 non-null object


In [53]:
df_procura_final.to_csv('resultados/empresas_de_politicos_maio_2019.csv', index=False)